# Deploy Orb Model Package from AWS Marketplace 


---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook. 

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-2/aws_marketplace|curating_aws_marketplace_listing_and_sample_notebook|ModelPackage|Sample_Notebook_Template|title_of_your_product-Model.ipynb)

---



Orb and the description: Orb is a universal interatomic potential for atomistic modelling of materials. Orb models are 3-6 times faster than existing universal potentials, stable under simulation for a range of out of distribution materials and, upon release, represented a 31% reduction in error over other methods on the Matbench Discovery benchmark.

This sample notebook shows you how to deploy [Orb](https://aws.amazon.com/marketplace/pp/prodview-ysg3nhoa7sewu) using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

## Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to [Orb](https://aws.amazon.com/marketplace/pp/prodview-ysg3nhoa7sewu). If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)

## Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Visualize output](#D.-Visualize-output)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Perform batch inference](#3.-Perform-batch-inference) 
4. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page [Orb](https://aws.amazon.com/marketplace/pp/prodview-ysg3nhoa7sewu)
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [ ]:
model_package_arn = "arn:aws:sagemaker:us-east-1:865070037744:model-package/orbmodelpackage-3fcf0a9dd0d838a69043e94193f8bff2"

In [ ]:
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import ModelPackage, get_execution_role
import boto3
import json
import requests
from botocore.auth import SigV4Auth
from botocore.awsrequest import AWSRequest
from botocore.exceptions import ClientError

In [ ]:
def create_or_get_sagemaker_role(role_name: str):
    iam_client = boto3.client('iam')
    
    # This is the trust policy allowing SageMaker to assume the role
    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "sagemaker.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }

    try:
        # Check if the role already exists
        response = iam_client.get_role(RoleName=role_name)
        print(f"Role '{role_name}' already exists. No action needed.")
        return response['Role']['Arn']
    except iam_client.exceptions.NoSuchEntityException:
        # The role does not exist, so create it
        print(f"Role '{role_name}' not found. Creating...")
        
        response = iam_client.create_role(
            RoleName=role_name,
            AssumeRolePolicyDocument=json.dumps(assume_role_policy_document),
            Description="Role for SageMaker to create/modify resources",
        )
        
        # Attach the AmazonSageMakerFullAccess policy
        iam_client.attach_role_policy(
            RoleName=role_name,
            PolicyArn="arn:aws:iam::aws:policy/AmazonSageMakerFullAccess"
        )
        
        print(f"Role '{role_name}' has been created and policy attached.")
        return response['Role']['Arn']
    except ClientError as e:
        print(f"Unexpected error: {e}")
        raise

In [ ]:
region = 'us-east-1'  # Specify your desired region here
boto3.setup_default_session(region_name=region)
# Pass the boto3 session into the SageMaker Session.
sagemaker_session = sage.Session()

# Use this if running in Amazon SageMaker Notebook Instance or on an EC2 instance
# Otherwise, create a role with the necessary permissions and pass it into the SageMaker Session manually.
try:
    role = get_execution_role(sagemaker_session)
    print(f"Execution role found: {role}")
except Exception as e:
    print(f"Could not retrieve execution role from SageMaker session. Reason: {e}")
    # Fallback: either use a known role or create one with boto3
    role = create_or_get_sagemaker_role("orb-model-sagemaker-role")

runtime = boto3.client("runtime.sagemaker")


## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [6]:
model_name = "Orb"
content_type = "application/octet-stream"
real_time_inference_instance_type = "ml.p3.2xlarge"

### A. Create an endpoint

In [ ]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session, 
)
# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

Once endpoint has been created, you would be able to perform real-time inference.

### B. Create input payload

Use Example Cif

In [8]:
file_name = "examples/4A.cif"
output_file_name = "output.json"

### C. Perform real-time inference

In order to invoke the endpoint, we use a signed request, using the requests library and AWS sigv4 authentication.
This uses your AWS credenetials to authenticate to the Sagemaker endpoint used in the request.

In [ ]:
def sigv4_request(
  url,
  method='GET',
  body=None,
  params=None,
  headers=None,
  service='sagemaker',
  region=sagemaker_session.boto_region_name,
  credentials=boto3.Session().get_credentials().get_frozen_credentials()
):
  """Sends an HTTP request signed with SigV4
  Args:
    url: The request URL (e.g. 'https://www.example.com').
    method: The request method (e.g. 'GET', 'POST', 'PUT', 'DELETE'). Defaults to 'GET'.
    body: The request body (e.g. json.dumps({ 'foo': 'bar' })). Defaults to None.
    params: The request query params (e.g. { 'foo': 'bar' }). Defaults to None.
    headers: The request headers (e.g. { 'content-type': 'application/json' }). Defaults to None.
    service: The AWS service name. Defaults to 'execute-api'.
    region: The AWS region id. Defaults to the env var 'AWS_REGION'.
    credentials: The AWS credentials. Defaults to the current boto3 session's credentials.
  Returns:
     The HTTP response
  """
  # sign request
  req = AWSRequest(
    method=method,
    url=url,
    data=body,
    params=params,
    headers=headers
  )

  SigV4Auth(credentials, service, region).add_auth(req)
  req = req.prepare()

  # send request
  return requests.request(
    method=req.method,
    url=req.url,
    headers=req.headers,
    data=req.body
  )

In [14]:

url=f"https://runtime.sagemaker.{region}.amazonaws.com/endpoints/{model_name}/invocations" 

with open(file_name, 'rb') as file:
  # Read the file as binary
  body = file.read()
# Create the signed request with the file body
  response = sigv4_request(
    url,
    method='POST',
    body=body,
    headers={'Content-Type': content_type},
  )
with open(output_file_name, 'w') as file:
  json.dump(response.json(), file)



### E. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
model.sagemaker_session.delete_endpoint("Orb")
model.sagemaker_session.delete_endpoint_config("Orb")

## 4. Clean-up

### A. Delete the model

In [ ]:
model.delete_model()

### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

